# Loss and Accuracy over Epochs

In [ ]:
import glob
import matplotlib.pyplot as plt
import json
import os
from itertools import cycle


def loss_plot(xlim: (int, int) = None):
    # Use glob to find all json files in the specified directory
    file_list = glob.glob('results/metrics_*.json')

    # Create a color cycle iterator to assign unique colors
    color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

    # Plot the data
    plt.figure(figsize=(14, 7))

    # Initialize a dictionary to store colors for each label
    color_map = {}

    # Subplot for Loss
    plt.subplot(1, 2, 1)
    for file in file_list:
        # Extract model_name and label_name from the file name
        filename = os.path.basename(file)
        model_label = filename.replace('metrics_', '').rsplit(
            '.', 1)[0]  # Remove the 'metrics_' prefix and '.json' suffix

        # Check if we have already assigned a color, otherwise get the next color
        if model_label not in color_map:
            color_map[model_label] = next(color_cycle)

        # Read json content
        with open(file, 'r') as f:
            data = json.load(f)

        epochs = list(range(1, len(data['train_loss']) + 1))
        plt.plot(epochs, data['train_loss'],
                 label=f'Train Loss ({model_label})', color=color_map[model_label])
        plt.plot(epochs, data['val_loss'],
                 label=f'Val Loss ({model_label})', linestyle='--', color=color_map[model_label])

    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.xlim(xlim)

    # Subplot for Accuracy
    plt.subplot(1, 2, 2)
    for file in file_list:
        # Extract model_name and label_name from the file name
        filename = os.path.basename(file)
        model_label = filename.replace('metrics_', '').rsplit(
            '.', 1)[0]  # Remove the 'metrics_' prefix and '.json' suffix

        # Read json content
        with open(file, 'r') as f:
            data = json.load(f)

        epochs = list(range(1, len(data['train_acc']) + 1))
        plt.plot(epochs, data['train_acc'],
                 label=f'Train Acc ({model_label})', color=color_map[model_label])
        plt.plot(epochs, data['val_acc'],
                 label=f'Val Acc ({model_label})', linestyle='--', color=color_map[model_label])

    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.xlim(xlim)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
loss_plot()

In [ ]:
import glob
import matplotlib.pyplot as plt
import json
import os
import numpy as np

start_epoch = 25

file_list = glob.glob('results/metrics_*.json')
for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)

    print(file)
    print(int(np.argmin(data['val_loss'][start_epoch:]))+start_epoch)
    print(int(np.argmax(data['val_acc'][start_epoch:]))+start_epoch)

# Show cropped images

In [ ]:
from VisionDatasets import ContactDataset
from PIL import Image
import json
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torchvision

paths_json = os.path.join(os.getcwd(), 'settings/paths.json')
with open(paths_json, 'r') as json_file:
    paths = json.load(json_file)

data = pd.read_csv(paths['silicone']['labels'], header=0)[:5]

dataset = ContactDataset(
    images=[Image.open(image)
            for image in data['HPC_Path'].tolist()],
    labels=data['GT'].to_numpy(),
    transform=None)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=1024,
    pin_memory=True)

for image, label in dataloader:
    display(torchvision.transforms.ToPILImage()(torch.mean(image[:], dim=0)))